In [1]:
from torchvision import utils
from basic_fcn import *
from dataloader import *
from transfer import *
from utils import *
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time
import math
from tqdm import tqdm
import gc
import os
import pickle

In [2]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        torch.nn.init.xavier_uniform_(m.bias.data.view(m.bias.data.shape[0],1))
        #a = math.sqrt(3) * math.sqrt(2/m.bias.data.shape[0])
        #torch.nn.init._no_grad_uniform_(m.bias.data, -a, a)
        
        


In [8]:

    
def train(model, criterion, epochs, train_loader, val_loader, test_loader, use_gpu, name, debug=False):
    if debug:
        initMem = {}
        print("Initialized with")
        initMem = checkM(initMem)
        
        
        
        
    #Create non-existing logfiles
    logname = 'logfile_transfer.txt'
    logname_summary = 'logfile_transfer_summary.txt'    
    
    if os.path.exists('logfile_transfer.txt') == True:
        i = 1
        logname = 'logfile_transfer' + str(i) + '.txt'
        while os.path.exists('logfile_transfer' + str(i) + '.txt'):
            i+=1
            logname = 'logfile_transfer' + str(i) + '.txt'
        logname_summary = 'logfile_transfer' + str(i) + '_summary.txt'    

    print('Loading results to logfile: ' + logname)
    with open(logname, "a") as file:
        file.write("Lofile Transfer DATA: Validation Loss and Accuracy\n") 
    
    print('Loading Transfer Summary to : ' + logname_summary) 
    pickle_file = logname_summary[::-4] +'.pkl'
    
        
        
    optimizer = optim.Adam(model.parameters(), lr=5e-3)
    if use_gpu:
        device = torch.device("cuda:0")
        model = torch.nn.DataParallel(model)
        model.to(device)
    
    if debug:
        print("Use GPU:  ")
        initMem = checkM(initMem)
        
    
    val_loss_set = []
    val_acc_set = []
    val_iou_set = []

    training_loss = []
    
    
    # Early Stop criteria
    minLoss = 1e6
    minLossIdx = 0
    earliestStopEpoch = 10
    earlyStopDelta = 5
    for epoch in range(epochs):
        ts = time.time()
             
                  
        if debug:
            singleM = {}
            
        for iter, (inputs, tar, labels) in enumerate(train_loader):
            
            #print(inputs.shape)
            optimizer.zero_grad()
            #print("\n$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
            #startMem = checkM(startMem)
            #singleM = checkM(singleM)
            
            del tar
            
            if use_gpu:
                inputs = inputs.to(device)# Move your inputs onto the gpu
                labels = labels.to(device) # Move your labels onto the gpu
            
                
            outputs = model(inputs)
            del inputs
            loss = criterion(outputs, Variable(labels.long()))
            del labels
            del outputs
            
            if debug:
                print("\n**********************************************\nPost Loss")
                #backMem = checkM(backMem, True)
                singleM = checkM(singleM)
                #print("start vs back diff")
                #memDiff(startMem, backMem)
            loss.backward()
            loss = loss.item()
            
            if debug:
                print("\n**********************************************\nPost Backward")
                #postLossMem = checkM(postLossMem, True)
                #print("Post loss vs back diff")
                singleM = checkM(singleM)
                #memDiff(backMem, postLossMem)
                
            optimizer.step()
            
            if debug:
                print("\n**********************************************\nPost Step")
                singleM = checkM(singleM)
                #finalMem = checkM(finalMem, True)
                #memDiff(postLossMem, finalMem)
            

            if iter % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss))
                break
            
            if debug:    
                print("\n**********************************************\nFinal")
                singleM = checkM(singleM)
            
                print("\n^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n")
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        
        # calculate val loss each epoch
        val_loss, val_acc, val_iou = val(model, val_loader, criterion, use_gpu)
        val_loss_set.append(val_loss)
        val_acc_set.append(val_acc)
        val_iou_set.append(val_iou)

        training_loss.append(loss)
        
        
        with open(logname, "a") as file:
            file.write("writing!\n")
            file.write("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
            file.write("\n training Loss:   " + str(loss))
            file.write("\n Validation Loss: " + str(val_loss_set[-1]))
            file.write("\n Validation acc:  " + str(val_acc_set[-1]))
            file.write("\n Validation iou:  " + str(val_iou_set[-1]) + "\n ")
            
            

            
        
        # Early stopping
        if val_loss < minLoss:
            # Store new best
            torch.save(model, name)
            minLoss = val_loss
            minLossIdx = epoch
            
        # If passed min threshold, and no new min has been reached for delta epochs
        elif epoch > earliestStopEpoch and (epoch - minLossIdx) > earlyStopDelta:
            print("Stopping early at {}".format(minLossIdx))
            
        # TODO what is this for?
        #model.train()
        
        with open(logname_summary, "w") as file:
            file.write("Summary!\n")
            file.write("Stopped early at {}".format(minLossIdx))
            file.write("\n training Loss:   " + str(training_loss))        
            file.write("\n Validation Loss: " + str(val_loss_set))
            file.write("\n Validation acc:  " + str(val_acc_set))
            file.write("\n Validation iou:  " + str(val_iou_set) + "\n ")

        
        with open(pickle_file, 'wb') as f:  # Python 3: open(..., 'wb')
            pickle.dump([training_loss, val_loss_set, val_acc_set,val_iou_set], f)
            
            
        
    return val_loss_set, val_acc_set, val_iou_set


def val(model, val_loader, criterion, use_gpu):
    
    # set to evaluation mode 
    model.eval()

    softmax = nn.Softmax(dim = 1)
    
    loss = []
    pred = []
    acc = []
    
    IOU_init = False
    if use_gpu:
        device = torch.device("cuda:0")
        
        #model.to(device)
        
    for iter, (inputs, tar, labels) in tqdm(enumerate(val_loader)):
        
        if not IOU_init:
            IOU_init = True
            IOU = np.zeros((1,19))
        del tar
        
        if use_gpu:
            inputs = inputs.to(device)
            labels = labels.to(device)
            

            
        with torch.no_grad():   
            outputs = model(inputs)  
            del inputs
            loss.append(criterion(outputs, labels.long()).item())
            prediction = softmax(outputs) 
            del outputs
            acc.append(pixel_acc(prediction, labels))
            IOU = IOU + np.array(iou(prediction, labels))
            del prediction
            del labels
        
    
    acc = sum(acc)/len(acc)
    avg_loss = sum(loss)/len(loss) 
    IOU = IOU/iter  
    
    return avg_loss, acc, IOU      
       
    
    
    
def test(model, use_gpu):
    
    softmax = nn.Softmax(dim = 1)
    
    pred = []
    acc = []
    if use_gpu:
        device = torch.device("cuda:0")
        
        model.to(device)
    
    IOU_init = False
    for iter, (X, tar, Y) in enumerate(test_loader):
        
        if not IOU_init:
            IOU_init = True
            IOU = np.zeros((1,tar.shape[1]))
        
        if use_gpu:
            inputs = X.to(device)
            labels = Y.to(device)
        else:
            inputs, labels = X, Y
                    
        
        outputs = model(inputs)  
        
        prediction = softmax(outputs)
        acc.append(pixel_acc(prediction, labels))
        IOU = IOU + np.array(iou(prediction, Y))
        
    acc = sum(acc)/len(acc)        
    IOU = IOU/iter

    #Complete this function - Calculate accuracy and IoU 
    # Make sure to include a softmax after the output from your model
    
    return acc, IOU
    


In [9]:
class Reshape(nn.Module):
    def __init__(self, shape):
        super(Reshape, self).__init__()
        self.shape = shape
        
    def forward(self, x):
        news = self.shape.copy()
        news[0] =  x.shape[0]
        
        return x.view(*tuple(news))

def getTransferModel(n_class):
    
    decoder = nn.Sequential(
        Reshape([1,512,32,64]),
        
        nn.Sequential(
            nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
        ),
        
        nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            ),
            
        nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            ),
            
        nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            ),
        
            
        nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            ),

        nn.Conv2d(32, n_class, kernel_size=1)
        )
    decoder.apply(init_weights)
        

    
    model = models.resnet34(pretrained=True)
        
    for param in model.parameters():
        # False implies no retraining
        param.requires_grad=False
        
    del param

    model.avgpool = nn.Identity() 
        
    model.fc = decoder
    #print(model)
    return model

In [ ]:
def checkM(prev, q=False):
    out = {}
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                if obj.is_cuda and not q:
                    name = str(obj.size())
                    if name in out:
                        out[name] += 1
                    else:
                        out[name] = 1
                    
        except:
            pass
        
    for key in out:
        if key not in prev:
            print("new: " + key + " : " + str(out[key]))
        elif prev[key] != out[key]:
            #print("diff (new - old): " + key + " : " + str(out[key] - prev[key]))
            print("diff (new - old): " + key + " : " + str(out[key])+ " - " +str(prev[key]))
            
    for key in prev:
        if key not in out:
            print("dropped: " + key + " : " + str(prev[key]))
    return out

def memDiff(prev, out):
    for key in out:
        if key not in prev:
            print("new: " + key + " : " + str(out[key]))
        elif prev[key] != out[key]:
            print("diff (new - old): " + key + " : " + str(out[key] - prev[key]))
            
    for key in prev:
        if key not in out:
            print("dropped: " + key + " : " + str(prev[key]))

if __name__ == "__main__":
    batch_size = 3
    train_dataset = CityScapesDataset(csv_file='train.csv')
    val_dataset = CityScapesDataset(csv_file='val.csv')
    test_dataset = CityScapesDataset(csv_file='test.csv')
    train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          num_workers=8,
                          shuffle=True)
    val_loader = DataLoader(dataset=val_dataset,
                          batch_size=batch_size,
                          num_workers=8,
                          shuffle=True)
    test_loader = DataLoader(dataset=test_dataset,
                          batch_size=batch_size,
                          num_workers=4,
                          shuffle=True)
    
    
    epochs     = 100
    criterion = torch.nn.CrossEntropyLoss()
    # Fix magic number
    model = getTransferModel(34)
    
    
    epochs     = 100
    use_gpu = torch.cuda.is_available()
    train(model, criterion, epochs, train_loader, val_loader, test_loader, use_gpu, "Transfer")
    
    
    model.load_state_dict(torch.load('./save_param'))
    

Loading results to logfile: logfile_transfer10.txt
Loading Transfer Summary to : logfile_transfer10_summary.txt
epoch0, iter0, loss: 3.830095052719116
Finish epoch 0, time elapsed 19.315083742141724



0it [00:00, ?it/s]
1it [00:08,  8.39s/it]
2it [00:08,  6.05s/it]
3it [00:12,  5.38s/it]
4it [00:13,  3.94s/it]
5it [00:15,  3.33s/it]
6it [00:15,  2.51s/it]
7it [00:17,  2.33s/it]
8it [00:18,  1.81s/it]
9it [00:20,  1.81s/it]
10it [00:20,  1.44s/it]
11it [00:23,  1.91s/it]
12it [00:24,  1.55s/it]
13it [00:25,  1.52s/it]
14it [00:26,  1.21s/it]
15it [00:27,  1.29s/it]
16it [00:28,  1.08s/it]
17it [00:30,  1.32s/it]
18it [00:31,  1.14s/it]
19it [00:33,  1.54s/it]
20it [00:34,  1.24s/it]
21it [00:35,  1.29s/it]
22it [00:36,  1.06s/it]
23it [00:37,  1.19s/it]
24it [00:38,  1.01s/it]
25it [00:39,  1.14s/it]
26it [00:40,  1.02it/s]
27it [00:41,  1.20s/it]
28it [00:42,  1.00s/it]
29it [00:43,  1.12s/it]
30it [00:44,  1.02it/s]
31it [00:46,  1.18s/it]
32it [00:46,  1.00s/it]
33it [00:48,  1.11s/it]
34it [00:48,  1.04it/s]
35it [00:50,  1.18s/it]
36it [00:50,  1.01s/it]
37it [00:52,  1.17s/it]
38it [00:53,  1.01it/s]
39it [00:54,  1.18s/it]
40it [00:55,  1.00it/s]
41it [00:56,  1.17s/it]
42it 

epoch1, iter0, loss: 5.698641300201416
Finish epoch 1, time elapsed 15.795316457748413



0it [00:00, ?it/s]
1it [00:08,  8.20s/it]
2it [00:13,  7.39s/it]
3it [00:15,  5.63s/it]
4it [00:15,  4.10s/it]
5it [00:16,  3.23s/it]
6it [00:17,  2.41s/it]
7it [00:18,  2.09s/it]
8it [00:19,  1.64s/it]
9it [00:21,  1.63s/it]
10it [00:21,  1.35s/it]
11it [00:23,  1.43s/it]
12it [00:23,  1.18s/it]
13it [00:25,  1.25s/it]
14it [00:25,  1.05s/it]
15it [00:27,  1.19s/it]
16it [00:28,  1.04s/it]
17it [00:29,  1.18s/it]
18it [00:30,  1.03s/it]
19it [00:31,  1.18s/it]
20it [00:32,  1.00s/it]

In [6]:
import pickle
newfile = 'newfile999.pkl'
a = 5
b =99
c = 100
with open(newfile, 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([a,b,c], f)

In [7]:
with open(newfile, 'rb') as f:
    [aac, bbc, ccc] = pickle.load(f)
print(aac)
print(bbc)
print(ccc)

5
99
100
